# HW 3 - Analysis of the MNIST Digit Dataset

**Author**: Ewan Lister
Completed 04/19/2023

In this notebook we will perform an analysis of the MNIST data set using multiple techniques.

### 1. Do an SVD analysis of the digit images. You will need to reshape each image into a column vector and each column of your data matrix is a different image.

In [1]:
# imports
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [2]:
# define functions for flattening and reshaping images
# shapes grid into column
def column(arr):
    return arr.flatten().reshape(-1, 1)

# shapes column into grid
def grid(arr):
    return arr.reshape(32, 32)


